In [1]:
import importlib
import torch
import datafawn
importlib.reload(datafawn)

from pathlib import Path

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA not available, using CPU")


Using GPU: NVIDIA GeForce RTX 5070 Ti Laptop GPU


In [2]:
# =============== POSE ESTIMATOR =============== #
dlc_estimator = datafawn.DeepLabCutPoseEstimator(
    model_name='superanimal_quadruped',
    detector_name='fasterrcnn_resnet50_fpn_v2',
    hrnet_model='hrnet_w32',
    max_individuals=1,
    pcutoff=0.15,
    device=device
)

# =============== POSTPROCESSORS =============== #
rel_paws = ['front_left_paw_rel', 'front_right_paw_rel', 'back_left_paw_rel', 'back_right_paw_rel']
reference_map = {
    'back_base': ['front_left_paw', 'front_right_paw'],
    'tail_base': ['back_left_paw', 'back_right_paw']
}

rel_pp = datafawn.RelativePawPositionPostprocessor()
error_pp = datafawn.ErrorPostprocessor(
    bodyparts=rel_paws,
    use_velocity=True,
    use_likelihood=True,
    use_distance=True,
    velocity_kwargs={'threshold_pixels': 50, 'window_size': 5},
    likelihood_kwargs={'min_likelihood': 0.5},
    distance_kwargs={'reference_map': reference_map, 'max_distance': 300}
)

# =============== EVENT EXTRACTOR =============== #
zeni_extractor = datafawn.ZeniExtractor(
    smooth_window_size=5,
    prominence_percentage=0.05,
    orientation_likelihood_threshold=0.0,
    orientation_smooth_window_size=15,
    show_plots=False
)

# =============== SOUNDSCAPE GENERATOR =============== #
SOUNDSCAPE_CONFIG = {
    'event_sound_map': {
        'front_left_paw_strike': Path('sounds/409837__datafawn__digital/838432__silverillusionist__glitch-faulty-core-2.wav'),
        'front_right_paw_strike': Path('sounds/409837__datafawn__digital/839817__silverillusionist__glitch-faulty-core-5.wav'),
        'back_left_paw_strike': Path('sounds/409837__datafawn__digital/839954__kerdwyn__single-note-d.wav'),
        'back_right_paw_strike': Path('sounds/409837__datafawn__digital/840060__pedr01__notification.wav')
    },
    'backing_track': Path('sounds/839814__apintofmild__industrial-plant-atmos-2-of-4.wav'),
    'volume': {
        'backing_track': 0.10,
        'event_sounds': 0.65,
        'original_video': 4.0
    }
}
# SOUNDSCAPE_CONFIG = {
#     'event_sound_map': {
#         'front_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398494__anthousai__wind-chimes-single-01.wav'),
#         'front_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398493__anthousai__wind-chimes-single-02.wav'),
#         'back_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398492__anthousai__wind-chimes-single-03.wav'),
#         'back_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398496__anthousai__wind-chimes-single-04.wav')
#     },
#     'backing_track': Path('data_example/sounds/22415__anthousai__wind-chimes/398495__anthousai__wind-chimes-rough.wav'),
#     'volume': {
#         'backing_track': 0.10,
#         'event_sounds': 0.65,
#         'original_video': 4.0
#     }
# }
ss_generator = datafawn.SoundScapeFromConfig(soundscape_config=SOUNDSCAPE_CONFIG)

ss_auto = datafawn.SoundScapeAuto()

In [3]:
# Create the FULL pipeline with all components
pipeline = datafawn.EventDetectionPipeline(
    pose_estimator=dlc_estimator,
    postprocessors=[rel_pp, error_pp],
    event_extractors=[zeni_extractor],
    # soundscape_generators=[ss_generator]
    soundscape_generators=[ss_auto]
)

In [4]:
# # Run full pipeline from raw video
# RAW_VIDEO_PATH = 'data/raw_videos/inspo_tiktok.mov'
# OUTPUT_DIR = 'data/tiktok'

# results = pipeline.run(
#     video_path=RAW_VIDEO_PATH,
#     output_dir=OUTPUT_DIR,
#     soundscape_input_video="pose_est"  # Use the labeled video for soundscape
# )

In [5]:
# Partial pipeline after events are extracted
results = pipeline.run(
    events_path='data/tiktok/events/events.json',
    output_dir='data/tiktok/',
    soundscape_input_video='data/tiktok/pose_estimation/inspo_tiktok_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4'
)

Loading video: data\tiktok\pose_estimation\inspo_tiktok_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4
Video FPS: 30.0
Video duration: 28.07 seconds

Found 12 back feet note files: ['C5.wav', 'D5.wav', 'E5.wav', 'F5.wav', 'G5.wav', 'A5.wav', 'B5.wav', 'C6.wav', 'D6.wav', 'E6.wav', 'F6.wav', 'G6.wav']
Found 12 front feet note files: ['C6.wav', 'D6.wav', 'E6.wav', 'F6.wav', 'G6.wav', 'A6.wav', 'B6.wav', 'C7.wav', 'D7.wav', 'E7.wav', 'F7.wav', 'G7.wav']
Speed array calculated: 740 frames

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing back_right_paw_strike:
    Found 61 strikes
    Strike timestamps: ['1.17s', '2.10s', '2.80s']...


OSError: Error in file sounds\custom_tone\B5.wav, Accessing time t=1.00-1.00 seconds, with clip duration=0.300000 seconds, 